In [1]:
import json
import spacy
import progressbar
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
import pickle

DATA_FILE1 = '../data/nouns_list.pkl'
DATA_FILE2 = '../data/pos_word_list.pkl'
DATA_FILE3 = '../data/neu_word_list.pkl'
DATA_FILE4 = '../data/neg_word_list.pkl'
DATA_FILE5 = '../data/others_list.pkl'
DATA_FILE6 = '../data/verbs_list.pkl'

train_list = []
dev_list = []
test_list = []

print("Reading from csv and splitting")
for i, line in enumerate(open('../data/reviews200k.json', 'r')):
    if i < 100000 and len(line) > 300:
        train_list.append(json.loads(line))
    if 99999 < i < 100500 and len(line) > 300:
        dev_list.append(json.loads(line))
    if 149999 < i < 200000 and len(line) > 300:
        test_list.append(json.loads(line))

Reading from csv and splitting


In [2]:
train_reviews = []
for i in range(0, 6):
    j = 0
    for line in train_list:
        if i == int(line['stars']):
            train_reviews.append(line)
            j = j + 1
        if j > 10:
            break

train_examples = [review['text'][0:100].lower() for review in train_reviews]
train_labels = [int(review['stars']) for review in train_reviews]

In [3]:
print(train_examples[0])
print(train_examples[1])

total bill for this horrible service? over $8gs. these crooks actually had the nerve to charge us $6
today was my second out of three sessions i had paid for. although my first session went well, i cou


In [4]:
spacy_nlp = spacy.load('en_core_web_sm')
sid = SentimentIntensityAnalyzer()

In [5]:
def set_polarity(word):
    if (sid.polarity_scores(word)['compound']) >= 0.1:
        return 'POS_JJ'
    elif (sid.polarity_scores(word)['compound']) <= -0.1:
        return 'NEG_JJ'
    else:
        return 'NEU_JJ'

In [6]:
tokenized_train_examples = []
train_examples_pos = []

for example in progressbar.progressbar(train_examples):
    doc = spacy_nlp(example)
    
    pos_example = []
    tokenized_example = []
    
    for token in doc:
        if token.pos_ not in 'PUNCT':
            if token.pos_ == 'ADJ':
                pos_example.append(set_polarity(token.text))
                tokenized_example.append(token.text)
            else:
                pos_example.append(token.tag_)
                tokenized_example.append(token.text)
                
    train_examples_pos.append(pos_example)
    tokenized_train_examples.append(tokenized_example)

100% (55 of 55) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [11]:
print(tokenized_train_examples[0])

['total', 'bill', 'for', 'this', 'horrible', 'service', 'over', '$', '8gs', 'these', 'crooks', 'actually', 'had', 'the', 'nerve', 'to', 'charge', 'us', '$', '6']


In [10]:
print(train_examples_pos[0])

['NEU_JJ', 'NN', 'IN', 'DT', 'NEG_JJ', 'NN', 'IN', '$', 'NN', 'DT', 'NNS', 'RB', 'VBD', 'DT', 'NN', 'TO', 'VB', 'PRP', '$', 'CD']


In [7]:
from parsing.text_base import rules_review
from parsing.text_base import rules_review
from parsing.grammar import Grammar

grammar = Grammar(rules_review)

Created grammar with 4 rules


In [8]:
from parsing.grammar import add_rule
from parsing.rule import Rule, rule_not_in_list

rules = []
rules_tag = []

for train_example_pos, tokenized_example in progressbar.progressbar(zip(train_examples_pos, tokenized_train_examples)):
    for pos, text in zip(train_example_pos, tokenized_example):
        rule = Rule('$' + pos, text)
        rule_tag = Rule('$Optional', '$' + pos)
        if rule_not_in_list(rule, rules):
            rules.append(rule)
        if rule_not_in_list(rule_tag, rules_tag):
            rules_tag.append(rule_tag)

| |#                                                 | 54 Elapsed Time: 0:00:00


In [9]:
def generate_sequence(random_number, lenght):
    if length == 2:
        return [random_number, random_number + 1, random_number + 2]
    elif length == 3:
        return [random_number, random_number + 1, random_number + 2]
    else:
        return [random_number, random_number + 1, random_number + 2, random_number + 4]

In [10]:
import random

def generate_random_sequences(sentence_length, random_length):
    sequences = []
    for i in range(2, 4):
        random_number = random.choice(range(0, sentence_length - 2))
        sequences.append(generate_sequence(random_number, i))
    return sequences

In [11]:
for train_example_pos, tokenized_example in progressbar.progressbar(zip(train_examples_pos, tokenized_train_examples)):
    sequences = generate_random_sequences(len(train_example_pos), 3, 2)
    print(sequences)
    for sequence in sequences:
        rhs = '$' + train_example_pos[sequence[0]]
        sem = '' + train_example_pos[sequence[0]]
        for i in range(1,len(sequence)):
            rhs = rhs + ' $' + train_example_pos[sequence[i]]
            sem = sem + ' ' + train_example_pos[sequence[i]]
        rule = Rule('$ReviewPattern', rhs, sem)
        rules.append(rule)

| |#                                                 | 54 Elapsed Time: 0:00:00


[[16, 17, 18], [3, 4, 5]]
[[16, 17, 18], [4, 5, 6]]
[[3, 4, 5], [5, 6, 7]]
[[10, 11, 12], [9, 10, 11]]
[[1, 2, 3], [10, 11, 12]]
[[12, 13, 14], [7, 8, 9]]
[[15, 16, 17], [14, 15, 16]]
[[15, 16, 17], [8, 9, 10]]
[[11, 12, 13], [12, 13, 14]]
[[7, 8, 9], [10, 11, 12]]
[[14, 15, 16], [9, 10, 11]]
[[2, 3, 4], [9, 10, 11]]
[[13, 14, 15], [13, 14, 15]]
[[4, 5, 6], [5, 6, 7]]
[[8, 9, 10], [6, 7, 8]]
[[13, 14, 15], [13, 14, 15]]
[[3, 4, 5], [5, 6, 7]]
[[17, 18, 19], [15, 16, 17]]
[[11, 12, 13], [13, 14, 15]]
[[21, 22, 23], [11, 12, 13]]
[[0, 1, 2], [3, 4, 5]]
[[13, 14, 15], [9, 10, 11]]
[[5, 6, 7], [8, 9, 10]]
[[11, 12, 13], [8, 9, 10]]
[[6, 7, 8], [0, 1, 2]]
[[16, 17, 18], [5, 6, 7]]
[[6, 7, 8], [7, 8, 9]]
[[7, 8, 9], [9, 10, 11]]
[[1, 2, 3], [8, 9, 10]]
[[18, 19, 20], [5, 6, 7]]
[[4, 5, 6], [10, 11, 12]]
[[10, 11, 12], [3, 4, 5]]
[[14, 15, 16], [6, 7, 8]]
[[13, 14, 15], [19, 20, 21]]
[[13, 14, 15], [7, 8, 9]]
[[13, 14, 15], [12, 13, 14]]
[[4, 5, 6], [9, 10, 11]]
[[12, 13, 14], [6, 7, 8]]
[[9,

In [12]:
for rule in rules:
    add_rule(grammar, rule)
for rule in rules_tag:
    add_rule(grammar, rule)

In [13]:
'''
from parsing.grammar import print_grammar

print_grammar(grammar)
'''

'\nfrom parsing.grammar import print_grammar\n\nprint_grammar(grammar)\n'

In [ ]:
import re

semantics_tuples = []

for example, label in progressbar.progressbar(zip(train_examples, train_labels)):
    s = re.sub(r'[^\w\s]','',example)

    parses = grammar.parse_input(s)
    
    semantics = []
    for parse in parses:
        if parse.semantics not in semantics:
            semantics.append(parse.semantics)
            
    for semantic in semantics:
        semantics_tuples.append((semantic, label))

| |        #                                          | 3 Elapsed Time: 0:00:50

In [ ]:
print(semantics_tuples)